In [1]:
import pandas as pd
import datetime

Agregamos los dataset para verificar la viabilidad de los objetivos de los KPI

Aclaración: Los KPIs se evaluarán inicialmente para determinar las metas objetivos que deben cumplir. Debido a la integralidad de los datos se utilizará desde el 2017-01-01 hasta el 2017-12-31

# Variación porcentual del volumen de ventas por mes

- **Área:** Ventas
    - **Objetivo:** Evaluar aumento o disminucion de la variación porcentual del volumen de ventas por mes
    - **Variables:**
      - $V_{actual}$ = volumen de ventas mes actual
      - $V_{anterior}$ = volumen de vental mes anterior

- **Fórmula:** $$VVV=\frac{V_{actual}-V_{anterior}}{V_{anterior}}\times 100 $$

  - **Frecuencia de evaluación:** Mensual
  - **Unidad:** Porcetaje [%]
  - **Valor objetivo:** 2 [%]


In [9]:
df_item = pd.read_csv("..\..\..\data_warehouse\datasets\olist_order_items_dataset.csv")

In [27]:
df_orders = pd.read_csv("..\..\..\data_warehouse\datasets\olist_orders_dataset.csv")

In [10]:
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [86]:
df_merged = pd.merge(df_item, df_orders) #Se unen los dos dataset para obtener la fecha de compra
DF = df_merged.drop(['shipping_limit_date', 'freight_value','order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis = 1)

In [87]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   order_id                  112650 non-null  object 
 1   order_item_id             112650 non-null  int64  
 2   product_id                112650 non-null  object 
 3   seller_id                 112650 non-null  object 
 4   price                     112650 non-null  float64
 5   customer_id               112650 non-null  object 
 6   order_status              112650 non-null  object 
 7   order_purchase_timestamp  112650 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.7+ MB


In [88]:
DF['order_status'].value_counts()

delivered      110197
shipped          1185
canceled          542
invoiced          359
processing        357
unavailable         7
approved            3
Name: order_status, dtype: int64

In [89]:
#Se eliminan las compras que se su estado sea 'unavailable' y 'canceled'
filter_orders = (DF['order_status'] != 'unavailable') & (DF['order_status'] != 'canceled')

In [90]:
merged_orders = DF[filter_orders].drop(['order_status','seller_id'],axis=1) #Se eliminan columnas que ya no se necesitan

In [91]:
merged_orders['order_purchase_timestamp']=pd.to_datetime(merged_orders['order_purchase_timestamp'])

In [92]:
merged_orders.columns

Index(['order_id', 'order_item_id', 'product_id', 'price', 'customer_id',
       'order_purchase_timestamp'],
      dtype='object')

In [98]:
merged_orders = merged_orders[(merged_orders['order_purchase_timestamp']>= datetime.datetime(2017,1,1)) & (merged_orders['order_purchase_timestamp']<= datetime.datetime(2017,12,31))]

In [99]:
merged_orders.sort_values('order_purchase_timestamp')

,order_id,order_item_id,product_id,price,customer_id,order_purchase_timestamp
104115,ec7a019261fce44180373d45b442d78f,1,f5d8f4fbc70ca2a0038b9a0010ed5cb0,10.90,c24fc5f9a446b4d8262041b9c64de772,2017-01-05 11:56:06
81567,b95a0a8bd30aece4e94e81f0591249d8,1,6c04a068e5ab37749c980c42a036b9e3,10.90,ff3c6d35b4927c9b965aa6a68dcf56e4,2017-01-05 12:01:20
24916,38bcb524e1c38c2c1b60600a80fc8999,1,680cc8535be7cc69544238c1d6a83fe8,2.90,d2c63ad286e3ca9dd69218008d61ff81,2017-01-05 12:06:36
53673,7a18a504c1a4b32d883e68de2e1a7db0,1,c0d4027067afcf9c1697cce981b8fed6,7.90,e6bee29442c761ff0dfb9a4414922a5c,2017-01-05 12:09:08
46997,6acecf438369055d9243e121045cca74,1,1514ddb0f4a5afc8d24104e89c714403,9.90,42d857e22f61f10476f0cda1aacc018a,2017-01-05 12:11:23
...,...,...,...,...,...,...
4782,0acce7378714d20548f0f5f1b71f6458,1,bcd58b852a3b0152b6b384c4e4474ba2,125.00,5b310d1f78f27de30f2fb3ddbc5e8a49,2017-12-30 21:48:24
1543,038eb192a216fe0c545a8ff3b1b96885,1,9548f3f06fa6c1983a2278e2a16ecdcb,39.90,320a107d49c46f919012be351ec06cbe,2017-12-30 22:07:13
92417,d1a5645d176964e7fc023cfc80d06f1a,1,553e0e7590d3116a072507a3635d2877,45.90,d3eb15c5980e25bf63acd83816968909,2017-12-30 22:29:45
5665,0cc76fbe09687fda664178e9fc6c404f,1,4c8b28305f570899b6ded964ddd234a9,89.90,007612fef51262f4762d32add5d128cc,2017-12-30 22:51:13


In [100]:
merged_orders = merged_orders.groupby('order_id')

In [101]:
df_item_ag = merged_orders.aggregate({'price':'sum', 'order_purchase_timestamp':'first'})

In [102]:
month_data = df_item_ag

In [103]:
month_data.set_index('order_purchase_timestamp', inplace = True)

In [104]:
df_month = month_data.resample('M').sum()

In [105]:
df_month

,price
order_purchase_timestamp,
2017-01-31,120098.27
2017-02-28,244959.35
2017-03-31,368341.32
2017-04-30,353842.98
2017-05-31,503159.19
2017-06-30,429916.61
2017-07-31,492287.30
2017-08-31,568245.79
2017-09-30,621415.91


In [106]:
df_month['dif_perc'] = df_month['price'].pct_change()

In [107]:
df_month['dif_perc'].map(lambda x:format(x,'.2%'))

order_purchase_timestamp
2017-01-31       nan%
2017-02-28    103.97%
2017-03-31     50.37%
2017-04-30     -3.94%
2017-05-31     42.20%
2017-06-30    -14.56%
2017-07-31     14.51%
2017-08-31     15.43%
2017-09-30      9.36%
2017-10-31      6.24%
2017-11-30     52.06%
2017-12-31    -26.93%
Freq: M, Name: dif_perc, dtype: object

In [108]:
df_month['dif_perc'].mean()

0.2260937179506587

Conclusión: Debido a la variación que se presenta entre meses en el valor de las ventas, se establece como objetivo el aumento del 10%

# Tasa de Convesión

Se agregan los dataset necesarios para evaluar el valor objetivo del KPI.

 - **Área:** Marketing
    - **Objetivo:** Medir la tasa de vendedores potenciales que se unen a la empresa
    - **Variables:**
      - $N°_{convertidos}$ = Número total de contactos convertidos
      - $N°_{interesados}$ = Número de contactos interesados
    - **Fórmula:** $$ TC=\left(\frac{N°_{convertidos}}{N°_{interesados}}\right)\times 100$$
    - **Frecuencia de evaluación:** Trimestral
    - **Unidad:** [%]
    - **Valor objetivo:** 15 [%]

In [10]:
df_marketing = pd.read_csv("..\..\..\data_warehouse\datasets\olist_marketing_qualified_leads_dataset.csv")
df_close_deals = pd.read_csv("..\..\..\data_warehouse\datasets\olist_closed_deals_dataset.csv")

In [112]:
df_marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [113]:
df_close_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mql_id                         842 non-null    object 
 1   seller_id                      842 non-null    object 
 2   sdr_id                         842 non-null    object 
 3   sr_id                          842 non-null    object 
 4   won_date                       842 non-null    object 
 5   business_segment               841 non-null    object 
 6   lead_type                      836 non-null    object 
 7   lead_behaviour_profile         665 non-null    object 
 8   has_company                    63 non-null     object 
 9   has_gtin                       64 non-null     object 
 10  average_stock                  66 non-null     object 
 11  business_type                  832 non-null    object 
 12  declared_product_catalog_size  69 non-null     flo

In [124]:
df_close_deals['won_date'] = pd.to_datetime(df_close_deals['won_date'])

In [125]:
df_close_deals = df_close_deals[(df_close_deals['won_date']>= datetime.datetime(2018,1,1)) & (df_close_deals['won_date']<= datetime.datetime(2018,12,31))]

In [127]:
df_close_deals['won_date'].max()

Timestamp('2018-11-14 18:04:19')

In [6]:
closed = df_close_deals["mql_id"]

In [7]:
#Se verifican que los vendedores con los que se cerró trato si se encuentren en la dataset del MQL
df_marketing[df_marketing['mql_id'].isin(closed)] 

,mql_id,first_contact_date,landing_page_id,origin
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search
12,a555fb36b9368110ede0f043dfc3b9a0,2018-04-04,2f08471dfb83baef522f952438a36f6e,referral
14,327174d3648a2d047e8940d7d15204ca,2018-04-03,1ceb590cd1e00c7ee95220971f82693d,organic_search
39,f5fee8f7da74f4887f5bcae2bafb6dd6,2018-01-14,22c29808c4f815213303f8933030604c,paid_search
67,ffe640179b554e295c167a2f6be528e0,2017-10-09,a0fbc0263cb98bdb0d72b9f9f283a172,unknown
...,...,...,...,...
7978,df9a2abe2ad3f2d8758b675ac8331ecf,2018-03-28,a7982125ff7aa3b2054c6e44f9d28522,organic_search
7983,8723201570415eb23b4f095d8c20a27c,2018-05-18,b76ef37428e6799c421989521c0e5077,organic_search
7991,2e57665b8faf05c967a801eb5aedfa0a,2018-05-25,40dec9f3d5259a3d2dbcdab2114fae47,paid_search
7994,40955be51cc85c5a5a6cdd12b19e9f10,2018-02-04,88740e65d5d6b056e0cda098e1ea6313,social


In [ ]:
df_marketing[]

In [64]:
N_interesados = df_marketing.shape[0]

In [65]:
N_convertidos = closed.shape[0]

In [66]:
TC = (N_convertidos/N_interesados) * 100

In [67]:
TC

10.525

Conclusión: Se evidencia que de los 8000 interesados en ser parte de Olist, solo 842 de estos se cerró trato, lo que equivale al 10.525%

# Puntuación neta del promotor

 - **Área:** Ventas
    - **Objetivo:** Medir la satisfacción del cliente
    - **Variables:**
      - Porcentaje de calificaciones positivas (> 3 estrellas)
      - Porcentaje de calificaciones negativas (<= 3 estrellas)
    - **Fórmula:** $$PN={{ \%}\ calificaciones\ positivas} - {{ \%}\ calificaciones\ negativas}$$
    - **Frecuencia de evaluación:** Mensual
    - **Unidad:** Porcentaje [%]
    - **Valor objetivo:** 30 [%]


In [9]:
df_reviews = pd.read_csv("..\..\..\data_warehouse\datasets\olist_order_reviews_dataset.csv")

In [10]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


Se agrega una columna para determinar los promotores (calificaciones mayores a 3) de los no promotores (calificaciones menores o iguales a 3 )

In [19]:
promotor = (df_reviews['review_score'] > 3)*1

In [20]:
df_reviews['promotor'] = promotor

In [53]:
no_promotor = df_reviews[df_reviews['promotor'] == 0].shape[0]

In [56]:
total_promotor = df_reviews[df_reviews['promotor'] == 1].shape[0]

In [57]:
t_cal = df_reviews.shape[0]

In [58]:
PN = ((total_promotor - no_promotor)/t_cal) * 100

In [59]:
PN

54.136096105780865

Conclusión: La puntuación neta del promotor actual se encuentra en 54.13% por lo cual se sugiere que el objetivo del KPI se aumente al 60%

# Fidelidad del cliente

 - **Área:** Ventas
    - **Objetivo:** Medir la tasa de clientes que vuelven a comprar dentro de un periodo determinado
    - **Variables:**
      - N° de clientes que volvieron a realizar compras
      - N° total de clientes
    - **Fórmula:** $$FC =\frac{N°\ de\ clientes\ que\ volvieron\ a\ comprar}{N°\ total\ de\ compra}\times 100$$
    - **Frecuencia de evaluación:** Trimestral
    - **Unidad:** [%]
    - **Valor objetivo:** 30 [%]

In [17]:
df_customers = pd.read_csv("..\..\..\data_warehouse\datasets\olist_customers_dataset.csv")
df_orders = pd.read_csv("..\..\..\data_warehouse\datasets\olist_orders_dataset.csv")

In [18]:
df_customers.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [19]:
df_orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [20]:
df_merged_FC = pd.merge(df_customers, df_orders, on='customer_id')

In [28]:
df_merged_FC.drop(columns=['customer_zip_code_prefix','customer_city', 'customer_state', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], inplace=True)

In [30]:
df_merged_FC['order_purchase_timestamp'] = pd.to_datetime(df_merged_FC['order_purchase_timestamp'])

In [41]:
df_merged_FC

,customer_id,customer_unique_id,order_id,order_status,order_purchase_timestamp
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33


In [42]:
df_merged_FC['quarter'] =df_merged_FC['order_purchase_timestamp']

In [38]:
df_quarter=  df_quarter.resample('Q')

In [39]:
df_quarter

# Puntualidad de la entrega

- **Área:** Logistica
  - **Objetivo:** Medir el porcentaje de entregas que se realizan a tiempo en relación con el número total de entregas.
  - **Variables:**
    - $N°_{entiempo}$ = Número de entregas a tiempo
    - $N°_{entregas}$ = Número total de entregas
  - **Fórmula:** $$ PE =\left(\frac{N°_{entiempo}}{N°_{entregas}}\right)\times 100$$
  - **Frecuencia de evaluación:** Mensual
  - **Unidad:** Porcentaje [%]
  - **Valor objetivo:** 95 [%]

In [11]:
df_puntualidad = pd.read_csv('..\..\..\data_warehouse\datasets\olist_orders_dataset.csv')

In [114]:
df_puntualidad[df_puntualidad['customer_id'] == '964253ff0e4e08180064764a450e521f']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
15791,c72727d29cde4cf870d569bf65edabfd,964253ff0e4e08180064764a450e521f,delivered,2017-02-07 18:01:15,2017-02-09 02:50:07,2017-02-10 05:32:55,2017-02-14 14:27:45,2017-07-04 00:00:00


In [116]:
df_puntualidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [117]:
df_puntualidad['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

Se evaluará solo los pedidos que ya se encuentran en estado 'delivered'

In [118]:
df_puntualidad[df_puntualidad['order_status']=='delivered']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


Teniendo en cuenta que para evaluar la puntualidad este ya debió ser entregado, solo tendremos en cuenta los datos que tengan 'order_status' en 'delivered'

In [76]:
df_puntualidad.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [78]:
import datetime

In [121]:
df_puntualidad['order_delivered_carrier_date'] = pd.to_datetime(df_puntualidad['order_delivered_carrier_date'])

In [122]:
df_puntualidad['order_delivered_customer_date'] = pd.to_datetime(df_puntualidad['order_delivered_customer_date'])

In [123]:
df_puntualidad['order_estimated_delivery_date'] = pd.to_datetime(df_puntualidad['order_estimated_delivery_date'])

In [124]:
df_puntualidad['order_purchase_timestamp'] = pd.to_datetime(df_puntualidad['order_purchase_timestamp'])

In [125]:
df_puntualidad['order_approved_at'] = pd.to_datetime(df_puntualidad['order_approved_at'])

In [126]:
df_puntualidad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96478 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96478 non-null  object        
 1   customer_id                    96478 non-null  object        
 2   order_status                   96478 non-null  object        
 3   order_purchase_timestamp       96478 non-null  datetime64[ns]
 4   order_approved_at              96464 non-null  datetime64[ns]
 5   order_delivered_carrier_date   96476 non-null  datetime64[ns]
 6   order_delivered_customer_date  96470 non-null  datetime64[ns]
 7   order_estimated_delivery_date  96478 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.6+ MB


In [132]:
df_puntualidad['puntual'] = (df_puntualidad['order_estimated_delivery_date'] > df_puntualidad['order_delivered_customer_date'])*1

In [133]:
df_puntualidad.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,puntual
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1


In [134]:
df_puntualidad['puntual'].value_counts()

1    88644
0     7834
Name: puntual, dtype: int64

In [135]:
N_entiempo = df_puntualidad[df_puntualidad['puntual']==1].shape[0]

In [137]:
N_entregas = df_puntualidad.shape[0]

In [139]:
PE = (N_entiempo / N_entregas)*100

In [140]:
PE

91.88001409647796

Conclusión: La puntualidad de entrega de los productos es de un 91.88%, por tanto, un objetivo del 95% es adecuado para el KPI